In [23]:
%%capture #忽略输出

import networkx as nx
import numpy as np

import Ipynb_importer

import s07_训练数据生成 as datagen

edges, nodes = datagen.load_data("edgeOSM.txt","nodeOSM.txt")
G = datagen.create_networkx_graph(edges, nodes)

def gen_full_edges(edge_ids):
    full_edges = []

    last_end_node_id = None
    last_edge_id = None

    for eid in edge_ids:
        eid = int(eid)
        if last_edge_id:
            if eid == last_edge_id :
                continue
        if last_end_node_id:
            curr_start_node_id = edges.loc[eid,'start_node']
            node_path = nx.shortest_path(G, source=last_end_node_id, target=curr_start_node_id)
            if len(node_path) >1 :
                for i in range(1,len(node_path)):
                    start_node = node_path[i-1]
                    end_node = node_path[i]
                    edge_gap = edges[edges['start_node']==start_node ][ edges['end_node']==end_node ]['eid'].item()
                    full_edges += [edge_gap]
        full_edges += [eid]
        last_end_node_id = edges.loc[eid,'end_node']
        last_edge_id = eid
    return full_edges

In [4]:
import nltk.translate.bleu_score as bleu

# 参考文本和翻译结果
ref_text = 'I love reading books'
hyp_text = 'I enjoy reading books'

# 分词
ref_tokens = ref_text.split()
hyp_tokens = hyp_text.split()

print(ref_tokens)
print(hyp_tokens)

# 计算BELU分数
bleu_1 = bleu.sentence_bleu([ref_tokens], hyp_tokens, weights=(1, 0, 0, 0))
bleu_2 = bleu.sentence_bleu([ref_tokens], hyp_tokens, weights=(0.5, 0.5, 0, 0))
bleu_3 = bleu.sentence_bleu([ref_tokens], hyp_tokens, weights=(0.33, 0.33, 0.33, 0))
bleu_4 = bleu.sentence_bleu([ref_tokens], hyp_tokens, weights=(0.25, 0.25, 0.25, 0.25))
belu_score = (bleu_1 + bleu_2 + bleu_3 + bleu_4) / 4

# 输出结果
print("BELU分数: ", belu_score)

['I', 'love', 'reading', 'books']
['I', 'enjoy', 'reading', 'books']
BELU分数:  0.3125


上述代码中使用nltk库中的`bleu_score`模块计算BELU分数，对于一个文本样本，计算出BLEU-1、BLEU-2、BLEU-3和BLEU-4的分数，并对其作平均，得到最终的BELU分数。

In [12]:
from sklearn.metrics import precision_recall_fscore_support as score

y_true = [1, 2, 3, 4, 5]
y_pred = [1, 2, 4, 4, 5]

precision, recall, fscore, support = score(y_true, y_pred)

print('Precision:', precision)
print('Recall:', recall)
print('F-score:', fscore)
print('Support:', support)

Precision: [1.  1.  0.  0.5 1. ]
Recall: [1. 1. 0. 1. 1.]
F-score: [1.         1.         0.         0.66666667 1.        ]
Support: [1 1 1 1 1]


C:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


这里我们使用了`sklearn.metrics`中的`precision_recall_fscore_support`进行计算precision、recall和F-score。

In [73]:
PREDICT =  ['1559', '304', '2591', '1561', '2991', '1691', '1689', '1689', '4251', '85', '1250', '84']
REAL    =['2993', '2993', '304', '1561', '2991', '1691', '1689', '4249', '4251', '1250', '84', '82']

In [74]:
# 分词
predict = gen_full_edges(PREDICT)
real = gen_full_edges(REAL)


ref_tokens = gen_full_edges(real)
hyp_tokens = gen_full_edges(predict)

print(hyp_tokens)
print(ref_tokens)


# 计算BELU分数
bleu_1 = bleu.sentence_bleu([ref_tokens], hyp_tokens, weights=(1, 0, 0, 0))
bleu_2 = bleu.sentence_bleu([ref_tokens], hyp_tokens, weights=(0.5, 0.5, 0, 0))
bleu_3 = bleu.sentence_bleu([ref_tokens], hyp_tokens, weights=(0.33, 0.33, 0.33, 0))
bleu_4 = bleu.sentence_bleu([ref_tokens], hyp_tokens, weights=(0.25, 0.25, 0.25, 0.25))
belu_score = (bleu_1 + bleu_2 + bleu_3 + bleu_4) / 4

print("BELU分数: ", belu_score)

[1559, 304, 4253, 2591, 2989, 1563, 1561, 2991, 1691, 1689, 4249, 4251, 85, 1250, 84]
[2993, 1559, 304, 4253, 2591, 2989, 1563, 1561, 2991, 1691, 1689, 4249, 4251, 85, 1250, 84, 82]
BELU分数:  0.8751733190429475


C:\Users\Administrator\AppData\Local\Temp\4\ipykernel_6624\399580839.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  edge_gap = edges[edges['start_node']==start_node ][ edges['end_node']==end_node ]['eid'].item()


In [75]:
from sklearn.metrics import precision_score, recall_score

list1 = real
list2 = predict


for i in range(len(list1)):
    for j in range(len(list2)):
        if list1[i] == list2[j]:
            print("第一个相同的数字为", list1[i], "，位置分别为", i, j)
            break
    else:
        continue
    break

list1 = list1[i:]
list2 = list2[j:]
    
    
# 求最短的序列长度
length = min(len(list1), len(list2))

# 切片截断序列
real = list1[:length]
predict = list2[:length]


precision = precision_score(real,predict,average='micro')

recall = recall_score(real,predict,average='micro')

print("Precision:", precision)

print("Recall:", recall)

def f_score(precision, recall, beta=1):
    beta_squared = beta ** 2
    numerator = (1 + beta_squared) * precision * recall
    denominator = beta_squared * precision + recall
    return numerator / denominator




# 计算 F1 score
f1_score = f_score(precision, recall)
print("F1 score:", f1_score)

# 计算 F2 score
f2_score = f_score(precision, recall, beta=2)
print("F2 score:", f2_score)

第一个相同的数字为 1559 ，位置分别为 1 0
Precision: 1.0
Recall: 1.0
F1 score: 1.0
F2 score: 1.0
